In [1]:
from contextlib import closing
import os
import platform
import socket
import subprocess
from subprocess import PIPE
import tempfile
import pandas as pd

from py4j.java_gateway import JavaGateway, GatewayParameters
from py4j.java_collections import ListConverter

In [2]:
__ERODE_JAR__ = "erodeMetrics.jar"#os.path.join(os.path.dirname(__file__), "erodeMetrics.jar")
def _start_server():
    # find a free port
    for port in range(25333, 65545):
        with closing(socket.socket(socket.AF_INET, socket.SOCK_STREAM)) as s:
            dest_addr = ("127.0.0.1", port)
            if s.connect_ex(dest_addr):
                break

    #ld_path = __ERODE_LIB_DIR__
    #java="java"
    java ="/Library/Java/JavaVirtualMachines/jdk-11.0.14.jdk/Contents/Home/bin/java"
    argv = [java, #f'-Djava.library.path="{ld_path}"',
                "-jar", __ERODE_JAR__, str(port)]

    if platform.system() == "Linux":
        env_ld_path = os.getenv("LD_LIBRARY_PATH")
        if env_ld_path:
            ld_path = f"{ld_path}:{env_ld_path}"
        env ={"LD_LIBRARY_PATH": ld_path}
        proc = subprocess.Popen(" ".join(argv), stdout=PIPE,
                shell=True, env=env)
    else:
        proc = subprocess.Popen(argv, stdout=PIPE)
    proc.stdout.readline()
    return proc, port

def _stop_server(proc):
    proc.terminate()
    try:
        proc.wait(5)
    except subprocess.TimeoutExpired:
        proc.kill()
        proc.wait()
        
def j_to_py_matrix(metrics_java):
    metrics_python= [ list(line) for line in metrics_java ]
    return metrics_python        

In [5]:
#_stop_server(_proc)
_proc, _port = _start_server()
gw_params = GatewayParameters(port=_port)#, auto_convert=True)
_gw = JavaGateway(gateway_parameters=gw_params)
erode = _gw.entry_point
_proc

<Popen: returncode: None args: ['/Library/Java/JavaVirtualMachines/jdk-11.0....>

In [6]:
erode.importAffine("test2.csv","test2B.csv");
nVars=erode.getVariablesNum()


In [4]:
# erode.loadModel("AM.ode");
# obtained = erode.computeBB();
# obtained
# print(erode.getPartitionString(obtained))

#a=erode.computeMetrics()
#for s in a:
#    print(s)

In [5]:
_port

25333

In [6]:
def loadERODESparseMatrix(AmatrixFile):
    Asparse=pd.read_csv(AmatrixFile)
    Asparse.columns=["row","column","A_{r,c}"]
    Asparse0=Asparse-1
    Asparse0["A_{r,c}"]=Asparse["A_{r,c}"]
    return Asparse0
Asparse=loadERODESparseMatrix("test2.csv")
Asparse

def loadERODEB(BFile):
    #pd.read_csv("test2B.csv")
    B=pd.read_csv(BFile,header=None)
    return B
B=loadERODEB("test2B.csv")
B

,0
0,0
1,0
2,0


In [7]:

erode.importAffine("test2.csv","test2B.csv");
nVars=erode.getVariablesNum()

modelString=erode.getModelString()
print(modelString)

obtained = erode.computeBB();
obtained
print(erode.getPartitionString(obtained))

metrics = erode.computeMetrics(obtained, 2.0);
for r in range(nVars):
    print()
    for c in range(nVars):
        print(metrics[r][c],end=" ")

test2.csv: 2 reactions, 3 species.
Species:
[S0, S1, S2]
 Reactions:
[S1-- (1.0) -->S0+S1, S2-- (1.0) -->S0+S2]
The partition has 2 blocks out of 3 species:
Block 1, Size: 1
0-S0 

Block 2, Size: 2
1-S1 
2-S2 


0.0 4.0 4.0 
4.0 0.0 0.0 
4.0 0.0 0.0 

In [9]:
def j_to_py_matrix(metrics_java):
    metrics_python= [ list(line) for line in metrics_java ]
    return metrics_python
metrics_python=j_to_py_matrix(metrics)
metrics_python

[[0.0, 4.0, 4.0], [4.0, 0.0, 0.0], [4.0, 0.0, 0.0]]

In [10]:
_stop_server(_proc)